In [4]:
import numpy as np
import pandas as pd
from vcimpute.helper_datagen import make_complete_data_matrix, mask_MCAR

In [5]:
X = make_complete_data_matrix(1000, 5, 'gaussian', seed=4)
X_mis = mask_MCAR(X, 'general', 0.2, seed=4)

In [ ]:
mdp = [False, False, False, False,  True]

In [53]:
n_miss_by_col(X_mis)

array([  0,   0, 103,   0, 297])

In [ ]:

np.equal(np.isnan(X_mis),)

np.logical_and(*)


mdps